In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# -*- coding: utf-8 -*-
"""
Paired slope plot + difference plot + effect sizes + bootstrap CI
+ corrected paired tests + Equivalence testing (TOST)

High-quality, journal-ready figures (svg + 600 dpi PNG).

Dependencies:
  pip install numpy pandas scipy statsmodels matplotlib
"""

from __future__ import annotations

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.weightstats import ttost_paired

from pathlib import Path

# -----------------------------
# 1) SETTINGS
# -----------------------------
element = "Sinusoids" # or "BC", "Nuclei", "Sinusoids", "Membranes"
OUT_PREFIX = element
OUT_DIR = Path("/content/drive/MyDrive/Manuscripts/LivesIsotropicReconstruction/data/results/figure2c/")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Equivalence bounds (choose a scientifically meaningful tolerance!)
# Example: within ±0.20 of the target is considered "equivalent"
EQ_BOUNDS_RAW = (-0.50, 0.50)   # for (Sim - Raw)
EQ_BOUNDS_ISO = (-0.50, 0.50)   # for (Sim - 1)

# Bootstrap settings
N_BOOT = 20000
SEED = 123


# Provide y-limits as (ymin, ymax). Use None to auto-scale.
YLIMS_SLOPE_RAW = (0.0, 3.5)     # for RAW slope plot
YLIMS_DIFF_RAW  = (-1.75, 1.75)    # for RAW diff plot (Sim - Raw)

OUT_DIR.mkdir(parents=True, exist_ok=True)

# -----------------------------
# 2) DATA (edit here or load CSV)
# -----------------------------
# Dictionary of datasets (decimal commas converted to Python floats with dots)
# Use like:
#   df = pd.DataFrame(DATASETS["Membranes"], columns=cols).set_index("SampleId")
#   ... then run your script as before.

DATASETS = {
    "Membranes": [
        [1, 1.806614033, 1.244329, 2.010325, 2.123197, 1.96224235, 1.056125304, 0.968543609, 0.951401791, 1.38136481],
        [2, 1.780158856, 1.587453, 2.310405, 2.364202, 1.90970349, 1.032280176, 1.340440986, 1.34422002,  1.41036202],
        [3, 2.794670776, 1.410881, 2.233726, 2.32189,  1.97548133, 1.207869547, 1.266338211, 1.264785327, 1.34391007],
        [4, 2.387366378, 1.335237, 2.197305, 2.27554,  2.08748669, 1.093783597, 1.290943347, 1.332411236, 1.47142883],
    ],
    "BC": [
        [1, 2.38091295,  1.507968645, 2.051961628, 2.077553513, 2.968303302, 1.122633657, 1.073217184, 1.067860149, 1.036504048],
        [2, 2.253584921, 1.354850543, 1.991388888, 2.059436193, 2.750387085, 0.828711681, 1.103971433, 1.090953536, 1.066579403],
        [3, 2.994040087, 1.493923585, 2.065334906, 2.100517469, 3.146904131, 1.171163186, 1.090762859, 1.080011643, 1.159330561],
        [4, 2.908995564, 1.799609654, 2.170871239, 2.16878252,  2.868644959, 1.261496875, 1.11370219,  1.11226693,  1.050318264],
    ],
    "Nuclei": [
        [1, 1.126918873, 1.314564988, 1.08521112,  0.988977551, 1.224655289, 1.335820202, 1.028610335, 0.965914229, 0.948928378],
        [2, 1.14368939,  1.285778147, 1.016605085, 0.954045326, 1.070651379, 1.277675588, 0.93764407,  0.931364408, 0.918829984],
        [3, 1.187160376, 1.320542097, 1.007440946, 0.964397061, 1.146398094, 1.317296317, 0.948804355, 0.950097115, 0.947777192],
        [4, 1.152685269, 1.311786251, 1.022775749, 0.970830479, 1.11808701,  1.270055121, 0.970462586, 0.965509973, 0.959806381],
    ],
    "Sinusoids": [
        [1, 1.595264445, 0.751947, 2.579401, 2.760093, 1.53925745, 0.875932074, 1.026441748, 0.965914229, 1.31619336],
        [2, 1.598817933, 1.106758, 2.394629, 2.556625, 1.47430931, 1.046493878, 1.112603131, 0.931364408, 1.31359842],
        [3, 1.634569875, 1.104118, 2.198634, 2.243178, 1.37310653, 1.118707708, 1.079191951, 0.950097115, 1.25130462],
        [4, 1.65411978,  0.936226, 2.539545, 2.784977, 1.48416626, 0.968415401, 1.132933665, 0.965509973, 1.322955],
    ],
}

cols = [
    "SampleId", "Raw",
    "CSim_SNR1", "CSim_SNR5", "CSim_SNR15", "GANSim",
    "CSimIsoRef_SNR1", "CSimIsoRef_SNR5", "CSimIsoRef_SNR15", "GAN_IsoRef",
]


df = pd.DataFrame(DATASETS[element], columns=cols).set_index("SampleId")

# -----------------------------
# 3) UTILITIES
# -----------------------------
def paired_cohens_dz(diff: np.ndarray) -> float:
    """Cohen's dz for paired designs using the difference vector."""
    diff = np.asarray(diff, dtype=float)
    sd = diff.std(ddof=1)
    return np.nan if sd == 0 else diff.mean() / sd


def wilcoxon_effect_r(stat: wilcoxon) -> float:
    """
    Approximate effect size r from Wilcoxon using normal approximation:
      r = |Z| / sqrt(n)
    SciPy doesn't return Z directly; we approximate via the p-value.
    """
    # Two-sided p -> |Z|
    p = stat.pvalue
    n = stat.statistic  # not n; ignore
    # Use inverse normal from p (approx). We'll use scipy if available; otherwise fallback.
    try:
        from scipy.stats import norm
        z = abs(norm.isf(p / 2.0))
    except Exception:
        # crude fallback
        z = abs(np.sqrt(2) * math.erfcinv(p))
    # Determine n from input length captured outside; this function expects caller to pass n
    return z  # caller will divide by sqrt(n)


def bootstrap_ci(diff: np.ndarray, func=np.mean, n_boot: int = 20000, seed: int = 0, ci: float = 0.95):
    """Bootstrap CI for a statistic (default: mean) on paired differences."""
    rng = np.random.default_rng(seed)
    diff = np.asarray(diff, dtype=float)
    n = diff.size
    idx = rng.integers(0, n, size=(n_boot, n))
    samples = diff[idx]
    stats = func(samples, axis=1)
    alpha = (1.0 - ci) / 2.0
    lo = np.quantile(stats, alpha)
    hi = np.quantile(stats, 1 - alpha)
    return float(lo), float(hi)


def holm_correct(pvals: list[float]) -> np.ndarray:
    """Holm correction (stronger than Bonferroni, standard for post-hoc)."""
    return multipletests(pvals, method="holm")[1]


def tost_equivalence_paired(x: np.ndarray, y: np.ndarray, low: float, high: float) -> dict:
    """
    Robust wrapper for statsmodels.ttost_paired
    Works across different statsmodels versions.
    """
    from statsmodels.stats.weightstats import ttost_paired

    x = np.asarray(x, float)
    y = np.asarray(y, float)

    res = ttost_paired(x, y, low, high)
    print(res)
    print(len(res))

    # Newer versions return 3 objects
    if len(res) == 3:
        pvalue, res_low, res_high = res
        t_low, p_low, df1 = res_low
        t_high, p_high, df2 = res_high
    else:
        # Older versions return 4 values
        t_low, p_low, t_high, p_high = res
        pvalue = max(p_low, p_high)

    return {
        "t_low": float(t_low),
        "p_low": float(p_low),
        "t_high": float(t_high),
        "p_high": float(p_high),
        "p_tost": float(pvalue),
    }


def journal_style():
    """Matplotlib style for clean, journal-like plots + editable text in Illustrator."""
    plt.rcParams.update({
        "figure.dpi": 150,
        "savefig.dpi": 600,

        # --- Fonts: use a common sans-serif that Illustrator handles well
        "font.family": "sans-serif",
        "font.sans-serif": ["Arial", "Helvetica", "DejaVu Sans"],

        "font.size": 9,
        "axes.titlesize": 10,
        "axes.labelsize": 9,
        "xtick.labelsize": 8,
        "ytick.labelsize": 8,
        "legend.fontsize": 8,
        "axes.linewidth": 0.8,
        "xtick.major.width": 0.8,
        "ytick.major.width": 0.8,

        # --- Critical: keep text as text (not paths)
        "text.usetex": False,
        "svg.fonttype": "none",   # keep SVG text as text
        "pdf.fonttype": 42,       # TrueType fonts in PDF (editable)
        "ps.fonttype": 42,        # TrueType fonts in PS/SVG (better, but  still flaky)

        # Avoid path simplification that can mess with text
        "path.simplify": False,
    })



def save_fig(fig, name: str):
    fig.tight_layout()

    pdf_path = OUT_DIR / f"{OUT_PREFIX}_{name}.svg"
    png_path = OUT_DIR / f"{OUT_PREFIX}_{name}.png"

    fig.savefig(pdf_path, bbox_inches="tight")
    fig.savefig(png_path, bbox_inches="tight", dpi=600)

# -----------------------------
# 4) CORE ANALYSIS (modify signature + body)
# -----------------------------
def analyze_vs_target(
    df: pd.DataFrame,
    target: np.ndarray,
    sim_cols: list[str],
    eq_bounds: tuple[float, float],
    title: str,
    panel_prefix: str,
    ylims_slope: tuple[float, float] | None = None,
    ylims_diff: tuple[float, float] | None = None,
):
    """
    Same analysis as before, but with explicit y-limits for:
      - slope plot (ylims_slope)
      - diff plot  (ylims_diff)
    """
    journal_style()
    n = df.shape[0]
    target = np.asarray(target, float)

    results = []
    pvals = []

    # Precompute diffs
    diffs = {}
    for col in sim_cols:
        sim = df[col].to_numpy(float)
        d = sim - target
        diffs[col] = d

        w = wilcoxon(d, alternative="two-sided", zero_method="wilcox", correction=False, mode="auto")
        pvals.append(w.pvalue)

        dz = paired_cohens_dz(d)
        try:
            from scipy.stats import norm
            z = abs(norm.isf(w.pvalue / 2.0))
        except Exception:
            z = np.nan
        r = (z / math.sqrt(n)) if np.isfinite(z) else np.nan

        mean_ci = bootstrap_ci(d, func=np.mean, n_boot=N_BOOT, seed=SEED)
        med_ci  = bootstrap_ci(d, func=np.median, n_boot=N_BOOT, seed=SEED + 1)

        tost = tost_equivalence_paired(df[col].to_numpy(float), target, eq_bounds[0], eq_bounds[1])

        results.append({
            "simulation": col,
            "n_animals": n,
            "mean_diff": float(np.mean(d)),
            "median_diff": float(np.median(d)),
            "abs_mean_diff": float(np.mean(np.abs(d))),
            "wilcoxon_stat": float(w.statistic),
            "p_wilcoxon": float(w.pvalue),
            "cohens_dz": float(dz),
            "wilcoxon_r": float(r),
            "boot95_mean_lo": mean_ci[0],
            "boot95_mean_hi": mean_ci[1],
            "boot95_median_lo": med_ci[0],
            "boot95_median_hi": med_ci[1],
            "tost_p": tost["p_tost"],
            "tost_p_low": tost["p_low"],
            "tost_p_high": tost["p_high"],
            "eq_low": eq_bounds[0],
            "eq_high": eq_bounds[1],
        })

    # Holm correction
    p_adj = holm_correct(pvals)
    for i in range(len(results)):
        results[i]["p_wilcoxon_holm"] = float(p_adj[i])

    res_df = pd.DataFrame(results).sort_values("abs_mean_diff", ascending=True)

    # -----------------------------
    # FIGURE A: paired slope plot
    # -----------------------------
    figA, axA = plt.subplots(figsize=(5.0, 2.5))
    x_positions = np.arange(len(sim_cols) + 1)
    labels = ["Target"] + sim_cols
    rng = np.random.default_rng(123)   # reproducible

    jitter_width = 0.06                # adjust if needed
    # jitter for each animal (same jitter used for all its points)
    jitter = (rng.random(n) - 0.5) * 2 * jitter_width

    axA.scatter(np.full(n, x_positions[0])+ jitter, target, s=25, zorder=3, marker="o")

    for i, col in enumerate(sim_cols, start=1):
        sim = df[col].to_numpy(float)

        axA.scatter(np.full(n, x_positions[i])+ jitter, sim, s=25, zorder=3, marker="o")
        for k in range(n):
            axA.plot([x_positions[0] + jitter[k], x_positions[i] + jitter[k]], [target[k], sim[k]], linewidth=0.5, alpha=0.5)

    #axA.set_xticks(x_positions)
    #axA.set_xticklabels(labels, rotation=35, ha="right")
    axA.set_xticks(x_positions)
    axA.set_xticklabels(list("abcde")[:len(x_positions)])

    axA.set_ylabel("Ax/Lat")
    #axA.set_title(f"{title} — Paired slope plot")

    # Apply explicit slope y-limits (if provided)
    axA.set_xlim((-0.5, 4.5))
    if ylims_slope is not None:
        axA.set_ylim(ylims_slope)

    # Annotation position derived from the final y-limits
    y0, y1 = axA.get_ylim()
    y_annot = y1 - 0.02 * (y1 - y0)

    for i, col in enumerate(sim_cols, start=1):
        p = res_df.loc[res_df["simulation"] == col, "p_wilcoxon_holm"].values[0]
        axA.text(x_positions[i], y_annot, f"p(Holm)={p:.3g}", ha="center", va="top")

    axA.set_yticklabels([])
    save_fig(figA, f"{panel_prefix}_A_slope")
    plt.close(figA)

    # -----------------------------
    # FIGURE B: difference plot (Sim - target)
    # -----------------------------
    figB, axB = plt.subplots(figsize=(5.0, 2.5))
    axB.axhline(0, linewidth=1.0)
    axB.axhspan(eq_bounds[0], eq_bounds[1], alpha=0.12)

    rng = np.random.default_rng(SEED)

    base_x = 2  # 'a' will be here (empty)
    xpos = base_x + 1 + np.arange(len(sim_cols))   # 3,4,5,6
    empty_x = base_x                                # 2

    for i, col in enumerate(sim_cols):
        d = diffs[col]
        x = np.full(n, xpos[i])
        jitter = (rng.random(n) - 0.5) * 0.12
        axB.scatter(x + jitter, d, s=28, marker="o", zorder=3)

        lo, hi = bootstrap_ci(d, func=np.mean, n_boot=N_BOOT, seed=SEED + i + 10)
        m = float(np.mean(d))
        axB.plot([xpos[i] - 0.18, xpos[i] + 0.18], [m, m], linewidth=2.0)
        axB.plot([xpos[i], xpos[i]], [lo, hi], linewidth=1.5)

        p_tost = res_df.loc[res_df["simulation"] == col, "tost_p"].values[0]
        axB.text(xpos[i], hi, f"TOST p={p_tost:.3g}", ha="center", va="bottom", fontsize=5)

    # ticks: a (empty), b,c,d,e for simulations
    xticks = np.r_[empty_x, xpos]                 # [2,3,4,5,6]
    xticklabels = list("abcde")

    axB.set_xticks(xticks)
    axB.set_xticklabels(xticklabels)

    axB.set_ylabel("Difference (Sim − Raw)")
    #axB.set_title(f"{title} — Difference plot (mean ± boot95% CI)")

    # Give breathing space on both sides
    #axB.set_xlim(base_x - 1, xpos[-1] + 1)
    axB.set_xlim(base_x - 0.5, xpos[-1] + 0.5)
    if ylims_diff is not None:
        axB.set_ylim(ylims_diff)

    axB.set_yticklabels([])
    save_fig(figB, f"{panel_prefix}_B_diff")
    plt.close(figB)

    return res_df

# -----------------------------
# 5) RUN THE TWO VALIDATIONS YOU DESCRIBED
# -----------------------------
# A) "Best reproduces Raw" (Target = Raw)
raw_target = df["Raw"].to_numpy(float)
sim_vs_raw = ["CSim_SNR1", "CSim_SNR5", "CSim_SNR15", "GANSim"]

res_raw = analyze_vs_target(
    df=df,
    target=raw_target,
    sim_cols=sim_vs_raw,
    eq_bounds=EQ_BOUNDS_RAW,
    title="Reproducing Raw",
    panel_prefix="RAW",
    ylims_slope=YLIMS_SLOPE_RAW,
    ylims_diff=YLIMS_DIFF_RAW,
)

# B) "IsoRef should match ideal value 1" (Target = 1)
iso_target = np.ones(df.shape[0], dtype=float)
sim_vs_iso = ["CSimIsoRef_SNR1", "CSimIsoRef_SNR5", "CSimIsoRef_SNR15", "GAN_IsoRef"]

res_iso = analyze_vs_target(
    df=df,
    target=iso_target,
    sim_cols=sim_vs_iso,
    eq_bounds=EQ_BOUNDS_ISO,
    title="Recovering ideal isotropy (target=1)",
    panel_prefix="ISO",
    ylims_slope=YLIMS_SLOPE_RAW,
    ylims_diff=YLIMS_DIFF_RAW,
)

# -----------------------------
# 6) SAVE SUMMARY TABLES
# -----------------------------
res_raw.to_csv(OUT_DIR /f"{OUT_PREFIX}_summary_vs_raw.csv", index=False)
res_iso.to_csv(OUT_DIR /f"{OUT_PREFIX}_summary_vs_iso.csv", index=False)

print("\n=== Ranking (closest to target) — vs Raw ===")
print(res_raw[["simulation", "abs_mean_diff", "mean_diff", "boot95_mean_lo", "boot95_mean_hi",
               "p_wilcoxon_holm", "tost_p"]].to_string(index=False))

print("\n=== Ranking (closest to target) — vs ideal 1 ===")
print(res_iso[["simulation", "abs_mean_diff", "mean_diff", "boot95_mean_lo", "boot95_mean_hi",
               "p_wilcoxon_holm", "tost_p"]].to_string(index=False))

print("\nSaved figures:")
print(f"  {OUT_PREFIX}_RAW_A_slope.eps/.png")
print(f"  {OUT_PREFIX}_RAW_B_diff.eps/.png")
print(f"  {OUT_PREFIX}_ISO_A_slope.eps/.png")
print(f"  {OUT_PREFIX}_ISO_B_diff.eps/.png")
print("\nSaved tables:")
print(f"  {OUT_PREFIX}_summary_vs_raw.csv")
print(f"  {OUT_PREFIX}_summary_vs_iso.csv")


# -----------------------------
# 7) NOTES (IMPORTANT FOR METHODS)
# -----------------------------
# - Wilcoxon tests: H0 median(Sim-Target)=0, Holm-corrected across sims in each family (vs Raw / vs 1).
# - Bootstrap CI shown is for the MEAN difference; you also have median CI in the CSV.
# - TOST is parametric (paired t-based) and tests equivalence within bounds EQ_BOUNDS_*.
#   Choose EQ_BOUNDS_* based on scientific relevance (e.g., acceptable error margin in your descriptor).

(np.float64(0.9129659137064984), (np.float64(-1.7745507592982686), np.float64(0.9129659137064984), np.float64(3.0)), (np.float64(-13.934775105273449), np.float64(0.0004000803625620637), np.float64(3.0)))
3
(np.float64(0.979146461437787), (np.float64(14.56595050257073), np.float64(0.0003508358188776554), np.float64(3.0)), (np.float64(3.4244447576975037), np.float64(0.979146461437787), np.float64(3.0)))
3
(np.float64(0.9823345102500135), (np.float64(11.511119630322064), np.float64(0.0007037412248025605), np.float64(3.0)), (np.float64(3.656516173218515), np.float64(0.9823345102500135), np.float64(3.0)))
3
(np.float64(0.001998223586037984), (np.float64(8.055086320937342), np.float64(0.001998223586037984), np.float64(3.0)), (np.float64(-15.157289798479741), np.float64(0.00031175425198779686), np.float64(3.0)))
3
(np.float64(0.0012199278254164875), (np.float64(9.635664369306616), np.float64(0.0011863360848038496), np.float64(3.0)), (np.float64(-9.544090046864099), np.float64(0.00121992782541

In [ ]:
from __future__ import annotations

import math
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.weightstats import ttost_paired


# =============================================================================
# 1) SETTINGS
# =============================================================================
OUT_PREFIX = "radius_validation"
OUT_DIR = Path("/content/drive/MyDrive/Manuscripts/LivesIsotropicReconstruction/data/results/figure2d")
OUT_DIR.mkdir(parents=True, exist_ok=True)

EQ_BOUNDS = (-0.2, 0.2)  # equivalence bounds for mean difference

N_BOOT = 20000
SEED = 123

YLIMS_SLOPE = (0.0, 1.75)
YLIMS_DIFF  = (-0.25, 0.65)


# =============================================================================
# 2) DATA (NEW ORDER)
# =============================================================================
# Min_raw | Avg_raw | Min_IdT | Avg_IdT
data = [
    [1.07833, 1.57522, 1.04385, 1.27235],
    [1.09092, 1.54434, 1.02233, 1.08341],
    [1.06669, 1.48987, 1.00339, 1.06661],
    [1.06605, 1.54784, 1.03111, 1.29864],
]

cols = ["Min_raw", "Avg_raw", "Min_IdT", "Avg_IdT"]
df = pd.DataFrame(data, columns=cols)


# =============================================================================
# 3) STATISTICAL UTILITIES
# =============================================================================
def paired_cohens_dz(diff):
    diff = np.asarray(diff, dtype=float)
    sd = diff.std(ddof=1)
    return np.nan if sd == 0 else diff.mean() / sd


def bootstrap_ci(diff, func=np.mean, n_boot=20000, seed=0, ci=0.95):
    rng = np.random.default_rng(seed)
    diff = np.asarray(diff, dtype=float)
    n = diff.size
    idx = rng.integers(0, n, size=(n_boot, n))
    stats = func(diff[idx], axis=1)
    alpha = (1.0 - ci) / 2.0
    return float(np.quantile(stats, alpha)), float(np.quantile(stats, 1 - alpha))


def holm_correct(pvals):
    return multipletests(pvals, method="holm")[1]


def tost_equivalence_paired(x, y, low, high):
    res = ttost_paired(x, y, low, high)

    if len(res) == 3:
        pvalue, res_low, res_high = res
        t_low, p_low, _ = res_low
        t_high, p_high, _ = res_high
    else:
        t_low, p_low, t_high, p_high = res
        pvalue = max(p_low, p_high)

    return float(pvalue)


def journal_style():
    plt.rcParams.update({
        "font.family": "sans-serif",
        "font.sans-serif": ["Arial", "Helvetica", "DejaVu Sans"],
        "pdf.fonttype": 42,
        "ps.fonttype": 42,
        "svg.fonttype": "none",
        "text.usetex": False,
        "savefig.dpi": 600,
        "font.size": 9,
    })


def save_fig(fig, name):
    fig.tight_layout()
    fig.savefig(OUT_DIR / f"{OUT_PREFIX}_{name}.pdf")
    fig.savefig(OUT_DIR / f"{OUT_PREFIX}_{name}.svg")
    fig.savefig(OUT_DIR / f"{OUT_PREFIX}_{name}.png", dpi=600)


# =============================================================================
# 4) ANALYSIS
# =============================================================================
def analyze_vs_target(df, target_col, comp_cols,
    ylims_slope: tuple[float, float] | None = None,
    ylims_diff: tuple[float, float] | None = None,):

    journal_style()

    target = df[target_col].to_numpy(float)
    n = len(target)

    results = []
    pvals = []
    diffs = {}

    for col in comp_cols:
        x = df[col].to_numpy(float)
        d = x - target
        diffs[col] = d

        w = wilcoxon(d)
        pvals.append(w.pvalue)

        mean_ci = bootstrap_ci(d, np.mean, N_BOOT, SEED)
        tost_p = tost_equivalence_paired(x, target, *EQ_BOUNDS)

        results.append({
            "comparison": col,
            "mean_diff": np.mean(d),
            "abs_mean_diff": np.mean(np.abs(d)),
            "boot_lo": mean_ci[0],
            "boot_hi": mean_ci[1],
            "p_wilcoxon": w.pvalue,
            "tost_p": tost_p,
        })

    p_adj = holm_correct(pvals)
    for i in range(len(results)):
        results[i]["p_wilcoxon_holm"] = p_adj[i]

    res_df = pd.DataFrame(results)

    # ============================
    # SLOPE PLOT
    # ============================
    figA, axA = plt.subplots(figsize=(6, 2.8))

    x_positions = np.arange(len(comp_cols) + 1)
    labels = [target_col] + comp_cols

    jitter = (np.random.rand(n) - 0.5) * 0.08

    axA.scatter(np.zeros(n)+jitter, target)

    for i, col in enumerate(comp_cols, 1):
        x = df[col].to_numpy(float)
        axA.scatter(np.full(n, i)+jitter, x)

        for k in range(n):
            axA.plot([0+jitter[k], i+jitter[k]], [target[k], x[k]], alpha=0.5)

        p = res_df.loc[res_df["comparison"] == col, "p_wilcoxon_holm"].values[0]
        #axA.text(i, max(target)*1.05, f"p={p:.3g}", ha="center")

    axA.set_xticks(x_positions)
    axA.set_xticklabels(labels, rotation=35, ha="right")
    axA.set_ylabel("Radius")

    if ylims_slope is not None:
        axA.set_ylim(ylims_slope)

    save_fig(figA, "A_slope")
    plt.close(figA)

    # ============================
    # DIFFERENCE PLOT
    # ============================
    figB, axB = plt.subplots(figsize=(6, 3))

    axB.axhline(0)
    axB.axhspan(*EQ_BOUNDS, alpha=0.15)

    for i, col in enumerate(comp_cols):
        d = diffs[col]
        jitter = (np.random.rand(n) - 0.5) * 0.12
        axB.scatter(np.full(n, i)+jitter, d)

        m = np.mean(d)
        lo, hi = bootstrap_ci(d, np.mean, N_BOOT, SEED+i)
        axB.plot([i-0.2, i+0.2], [m, m], linewidth=2)
        axB.plot([i, i], [lo, hi])

        p_tost = res_df.loc[res_df["comparison"] == col, "tost_p"].values[0]
        axB.text(i, hi, f"TOST={p_tost:.3g}", ha="center")

    axB.set_xticks(np.arange(len(comp_cols)))
    axB.set_xticklabels(comp_cols, rotation=35, ha="right")
    axB.set_ylabel("Difference (Comp − Min_raw)")
    if ylims_diff is not None:
        axB.set_ylim(ylims_diff)

    save_fig(figB, "B_diff")
    plt.close(figB)

    return res_df


# =============================================================================
# 5) RUN
# =============================================================================
res = analyze_vs_target(
    df,
    target_col="Min_raw",
    comp_cols=["Avg_raw", "Min_IdT", "Avg_IdT"],
    ylims_slope=YLIMS_SLOPE,
    ylims_diff=YLIMS_DIFF
)

print(res)


  comparison  mean_diff  abs_mean_diff   boot_lo   boot_hi  p_wilcoxon  \
0    Avg_raw   0.463820       0.463820  0.437833  0.489340       0.125   
1    Min_IdT  -0.050327       0.050327 -0.065945 -0.034710       0.125   
2    Avg_IdT   0.104755       0.108550 -0.003795  0.213305       0.625   

     tost_p  p_wilcoxon_holm  
0  0.999745            0.375  
1  0.000243            0.375  
2  0.114400            0.625  


In [ ]:
# -*- coding: utf-8 -*-
"""
Paired slope plot + difference plot + bootstrap CI
+ Wilcoxon (paired, Holm if multiple) + Equivalence testing (TOST)

Adapted for: Nuclei elongation (Raw vs IdT), n=4 pairs

Outputs: PDF + SVG (editable text) + 600 dpi PNG
"""

from __future__ import annotations

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.weightstats import ttost_paired


# =============================================================================
# 1) SETTINGS
# =============================================================================
OUT_PREFIX = "nuclei_elongation"
OUT_DIR = Path("/content/drive/MyDrive/Manuscripts/LivesIsotropicReconstruction/data/results/figure2e")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Equivalence bounds for paired differences: (IdT - Raw)
# IMPORTANT: choose based on what you consider "practically equivalent"
EQ_BOUNDS = (-0.05, 0.05)

N_BOOT = 20000
SEED = 123

# Optional y-limits (set None to auto)
YLIMS_SLOPE = (0.0, 0.25)
YLIMS_DIFF  = (-0.15, 0.15)


# =============================================================================
# 2) DATA (Raw | IdT)
# =============================================================================
data = [
    [0.185472, 0.0444513],
    [0.151785, 0.0350925],
    [0.200817, 0.0672715],
    [0.176964, 0.0500642],
]
cols = ["Raw", "IdT"]
df = pd.DataFrame(data, columns=cols)


# =============================================================================
# 3) UTILITIES
# =============================================================================
def bootstrap_ci(diff, func=np.mean, n_boot=20000, seed=0, ci=0.95):
    rng = np.random.default_rng(seed)
    diff = np.asarray(diff, dtype=float)
    n = diff.size
    idx = rng.integers(0, n, size=(n_boot, n))
    stats = func(diff[idx], axis=1)
    alpha = (1.0 - ci) / 2.0
    return float(np.quantile(stats, alpha)), float(np.quantile(stats, 1 - alpha))


def holm_correct(pvals):
    return multipletests(pvals, method="holm")[1]


def tost_equivalence_paired(x, y, low, high):
    """
    Returns the overall TOST p-value (the max of the two one-sided p-values)
    Compatible with different statsmodels versions.
    """
    res = ttost_paired(np.asarray(x, float), np.asarray(y, float), low, high)

    if len(res) == 3:
        pvalue, res_low, res_high = res
        # res_low/res_high are tuples: (t, p, df)
        # We return pvalue directly (already combined)
        return float(pvalue)
    else:
        t_low, p_low, t_high, p_high = res
        return float(max(p_low, p_high))


def journal_style():
    plt.rcParams.update({
        "font.family": "sans-serif",
        "font.sans-serif": ["Arial", "Helvetica", "DejaVu Sans"],
        "pdf.fonttype": 42,      # editable text in PDF
        "ps.fonttype": 42,
        "svg.fonttype": "none",  # keep text as text in SVG
        "text.usetex": False,
        "savefig.dpi": 600,
        "font.size": 9,
        "axes.linewidth": 0.8,
        "xtick.major.width": 0.8,
        "ytick.major.width": 0.8,
    })


def save_fig(fig, name):
    fig.tight_layout()
    fig.savefig(OUT_DIR / f"{OUT_PREFIX}_{name}.pdf", bbox_inches="tight")
    fig.savefig(OUT_DIR / f"{OUT_PREFIX}_{name}.svg", bbox_inches="tight")
    fig.savefig(OUT_DIR / f"{OUT_PREFIX}_{name}.png", bbox_inches="tight", dpi=600)


# =============================================================================
# 4) ANALYSIS (Raw is target, compare IdT)
# =============================================================================
def analyze_vs_target(
    df: pd.DataFrame,
    target_col: str,
    comp_cols: list[str],
    ylims_slope: tuple[float, float] | None = None,
    ylims_diff: tuple[float, float] | None = None,
):
    journal_style()

    target = df[target_col].to_numpy(float)
    n = len(target)

    results = []
    pvals = []
    diffs = {}

    for col in comp_cols:
        x = df[col].to_numpy(float)
        d = x - target  # (Comp - Target)
        diffs[col] = d

        w = wilcoxon(d)  # two-sided by default
        pvals.append(w.pvalue)

        mean_ci = bootstrap_ci(d, np.mean, N_BOOT, SEED)
        tost_p = tost_equivalence_paired(x, target, *EQ_BOUNDS)

        results.append({
            "comparison": col,
            "mean_diff": float(np.mean(d)),
            "abs_mean_diff": float(np.mean(np.abs(d))),
            "boot_lo": mean_ci[0],
            "boot_hi": mean_ci[1],
            "p_wilcoxon": float(w.pvalue),
            "tost_p": float(tost_p),
            "eq_low": EQ_BOUNDS[0],
            "eq_high": EQ_BOUNDS[1],
        })

    # Holm correction (here it is trivial because we compare only 1 column,
    # but kept for consistency if you add more comparisons later)
    p_adj = holm_correct(pvals)
    for i in range(len(results)):
        results[i]["p_wilcoxon_holm"] = float(p_adj[i])

    res_df = pd.DataFrame(results)

    # ============================
    # SLOPE PLOT (paired)
    # ============================
    figA, axA = plt.subplots(figsize=(4.6, 2.8))
    labels = [target_col] + comp_cols
    x_positions = np.arange(len(labels))

    rng = np.random.default_rng(SEED)
    jitter = (rng.random(n) - 0.5) * 0.08

    # target points
    axA.scatter(np.full(n, x_positions[0]) + jitter, target, s=28, zorder=3)

    # comparisons
    for i, col in enumerate(comp_cols, start=1):
        x = df[col].to_numpy(float)
        axA.scatter(np.full(n, x_positions[i]) + jitter, x, s=28, zorder=3)

        # connect paired points
        for k in range(n):
            axA.plot([x_positions[0] + jitter[k], x_positions[i] + jitter[k]],
                     [target[k], x[k]], alpha=0.5, linewidth=0.8)

        # annotate corrected p
        p = res_df.loc[res_df["comparison"] == col, "p_wilcoxon_holm"].values[0]
        # place annotation near top of axis
        y0, y1 = axA.get_ylim()
        axA.text(x_positions[i], y1, f"p(Holm)={p:.3g}", ha="center", va="top")

    axA.set_xticks(x_positions)
    axA.set_xticklabels(labels, rotation=0)
    axA.set_ylabel("Nuclei elongation")
    axA.set_title("Paired slope plot")

    if ylims_slope is not None:
        axA.set_ylim(ylims_slope)

    save_fig(figA, "A_slope")
    plt.close(figA)

    # ============================
    # DIFFERENCE PLOT (IdT - Raw)
    # ============================
    figB, axB = plt.subplots(figsize=(4.6, 3.0))

    axB.axhline(0, linewidth=1.0)
    axB.axhspan(*EQ_BOUNDS, alpha=0.15)

    for i, col in enumerate(comp_cols):
        d = diffs[col]
        rng = np.random.default_rng(SEED + 100 + i)
        jitter = (rng.random(n) - 0.5) * 0.12

        axB.scatter(np.full(n, i) + jitter, d, s=28, zorder=3)

        m = float(np.mean(d))
        lo, hi = bootstrap_ci(d, np.mean, N_BOOT, SEED + i)
        axB.plot([i - 0.2, i + 0.2], [m, m], linewidth=2.0)
        axB.plot([i, i], [lo, hi], linewidth=1.2)

        p_tost = res_df.loc[res_df["comparison"] == col, "tost_p"].values[0]
        axB.text(i, hi, f"TOST p={p_tost:.3g}", ha="center", va="bottom")

    axB.set_xticks(np.arange(len(comp_cols)))
    axB.set_xticklabels(comp_cols)
    axB.set_ylabel("Difference (IdT − Raw)")
    axB.set_title("Difference plot (mean ± boot95% CI)")

    if ylims_diff is not None:
        axB.set_ylim(ylims_diff)

    save_fig(figB, "B_diff")
    plt.close(figB)

    return res_df


# =============================================================================
# 5) RUN
# =============================================================================
res = analyze_vs_target(
    df=df,
    target_col="Raw",
    comp_cols=["IdT"],
    ylims_slope=YLIMS_SLOPE,
    ylims_diff=YLIMS_DIFF,
)

print(res)

print("\nSaved figures:")
print(f"  {OUT_PREFIX}_A_slope.pdf/.svg/.png")
print(f"  {OUT_PREFIX}_B_diff.pdf/.svg/.png")
print("\nSaved table columns:")
print(list(res.columns))

  comparison  mean_diff  abs_mean_diff  boot_lo   boot_hi  p_wilcoxon  \
0        IdT   -0.12954        0.12954 -0.13749 -0.120906       0.125   

     tost_p  eq_low  eq_high  p_wilcoxon_holm  
0  0.999703   -0.05     0.05            0.125  

Saved figures:
  nuclei_elongation_A_slope.pdf/.svg/.png
  nuclei_elongation_B_diff.pdf/.svg/.png

Saved table columns:
['comparison', 'mean_diff', 'abs_mean_diff', 'boot_lo', 'boot_hi', 'p_wilcoxon', 'tost_p', 'eq_low', 'eq_high', 'p_wilcoxon_holm']
